In [69]:
import serial
import crcmod
import struct
import binascii

# COM port configuration
RelayCOMPort = 'COM8'

# Serial communication needs to be done in Bytes for this ZS Relay Board not Hex

In [61]:
# Open serial port at com 8
def openSerial(COM: str):
    serialPort = serial.Serial(COM, 38400, timeout=1)
    return serialPort

RelayCOM = openSerial(RelayCOMPort)


In [60]:
# close serial port
def closeSerial(serialPort: str):
    serialPort.close()

closeSerial(RelayCOM)

In [86]:
# seiral port write in hex
def writeHexSerial(serialPort: str, command: str):
    hex_command = bytes.fromhex(command)
    serialPort.write(hex_command)
    response = serialPort.read(serialPort.in_waiting or 1)
    return response

# all relay on command
allRelayOn = '01 06 00 34 00 01 09 C4'
allRelayOff = '01 06 00 34 00 00 C8 04'

In [110]:
def writeByteSerial(serialPort: str, command: bytes):
    serialPort.write(command)
    response = serialPort.read(serialPort.in_waiting or 1)
    return response

In [87]:
response = writeHexSerial(RelayCOM, allRelayOn)
print(response)

b'\x01'


In [105]:
response = writeHexSerial(RelayCOM, allRelayOff)
print(response) 

b'\x06\x004\x00\x01\t\xc4'


In [98]:
# Use library to calculate CRC-16 Modbus for the last two byte of a command
command_1 = '01 06 00 34 00 01'

# Convert the command to bytes
command_bytes = bytes.fromhex(command_1.replace(" ", ""))

# Define the CRC-16 Modbus function
crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)

# Calculate the CRC
crc_result = crc16(command_bytes)

# reverse the bytes order
crc_result = struct.pack("<H", crc_result)

# command_1b = binascii.hexlify(crc_result)

# Print the result in hexadecimal
print(crc_result.hex())

09c4


In [101]:
# practice to use string based command to obtain crc
# combine the byte form of command with crc (byte form)
# send out the command

command_1_hex = bytes.fromhex(command_1)
print(command_1_hex)
full_command_1 = command_1_hex + crc_result
print(full_command_1)
response = writeHexSerial(RelayCOM, full_command_1.hex())

b'\x01\x06\x004\x00\x01'
b'\x01\x06\x004\x00\x01\t\xc4'


In [103]:
# practice with decimal, hex and bytes
pos_1 = 1
print(pos_1)
# bytes of pos_1
pos_1_bytes = pos_1.to_bytes(1, byteorder='big')
print(pos_1_bytes)
# note that the result is in hex \x01 instead of 1
# bytes and binary are different

1
b'\x01'


In [114]:
# breakout crc logic to a function
def crc16Modbus(command: bytes):
    crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)
    crc_result = crc16(command)
    crc_result = struct.pack("<H", crc_result)
    return crc_result

# test the function
# command_1 = '01 06 00 34 00 01'
command_1_hex = bytes.fromhex(command_1)
print(command_1_hex)
crc_result = crc16Modbus(command_1_hex)
print(crc_result)
full_command_1 = command_1_hex + crc_result
print(full_command_1.hex())

b'\x01\x06\x004\x00\x01'
b'\t\xc4'
01060034000109c4


In [119]:
# create a library of command to controll relay on and off
# default device position is 01 
# on and off action is 06
# relay address consist of 2 bytes i.e., 00 01 for channel 2
# the next 2 bytes are the on and off state i.e., 00 00 for off and 00 01 for on
# the last 2 bytes are the CRC-16 Modbus
# the function takes in hex string for device position, relay address and relay state
# the function returns a hex string command

def _switchCommand(devicePosition: int, relayAddress: int, relayState: int) -> bytes:
    devicePosition = devicePosition.to_bytes(1, byteorder='big')
    relayAddress = relayAddress.to_bytes(2, byteorder='big')
    relayState = relayState.to_bytes(2, byteorder='big')
    command = devicePosition + b'\x06' + relayAddress + relayState
    crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)
    crc_result = crc16(command)
    crc_result = struct.pack("<H", crc_result)
    full_command = command + crc_result
    return full_command



constr_command = relayCommand(1,1,0)

In [116]:
response = writeByteSerial(RelayCOM, constr_command)